# Analyze bicycle network results
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook takes the results from 03_poi_based_generation and 04_connect_clusters and calculates/analyzes a number of measures.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-07-14

## Preliminaries

### Parameters

In [ ]:
placeid = "copenhagen"
debug = True # If True, will produce plots and/or verbose output to double-check
prune_measure = "betweenness"
poi_source = "railwaystation" # railwaystation, grid, popdensity
buffer_walk = 500 # Buffer in m for coverage calculations. (How far people are willing to walk)
numnodepairs = 500 # Number of node pairs to consider for random sample to calculate directness (O(numnodepairs^2), so better not go over 1000)
parameterids = ["carall", "bikeable", "biketrack", "biketrackcarall"]

### Setup

In [70]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

Loaded PATH


=== Cities ===
{'budapest': {'countryid': 'hun', 'nominatimstring': 'Budapest, Hungary'}}


Setup finished

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
matplotlib 3.2.1
csv        1.0
networkx   2.4
osmnx      0.13.0
pyproj     2.6.1.post1
watermark  2.0.2
geopandas  0.7.0
pandas     1.0.3
fiona      1.8.13
numpy      1.18.4
shapely    1.7.0
igraph     0.8.2
Tue Jul 14 2020 

CPython 3.8.2
IPython 7.14.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit
Git hash   :


### Functions

In [66]:
%run -i functions.py

Loaded functions


## Load data

In [ ]:
placepath = PATH["data"] + placeid + "/"
G_carall = csv_to_ig(placepath, placeid, 'carall')

if poi_source == "railwaystation":
    with open(placepath + placeid + '_poi_railwaystation_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]

In [ ]:
filename = placeid + '_poi_' + poi_source + "_" + prune_measure
resultfile = open(PATH["results"] + filename + ".pickle",'rb')
res = pickle.load(resultfile)
resultfile.close()
if debug: pp.pprint(res)

## Analyze results

cost (length)  
length / carlength  
coverage  
directness  
compare with existing bike infra  

In [ ]:
# output_place is one static file for the existing city. This can be compared to the generated infrastructure.
# Make a check if this file was already generated. If not, generate it:

filename = placeid + '_existing.csv"
if not os.path.isfile(PATH["results"] + filename):
    empty_metrics = {
                     "length":0,
                     "coverage": 0,
                     "directness": 0,
                     "poi_coverage": 0,
                     "components": 0
                    }
    output_place = {}
    for parameterid in parameterids:
        output_place[parameterid] = copy.deepcopy(empty_metrics)
    output_place["general"] = {"poirailway": 0}
                   
                    
    # Load all networks
    Gs = {}
    for parameterid in parameterids:
        Gs[parameterid] = csv_to_ig(PATH, placeid, parameterid)
        metrics = calculate_metrics(Gs[parameterid], Gs[parameterid], nnids, buffer_walk, numnodepairs)
        for key, val in metrics:
            output_place[parameterid][key] = val
    # TO DO: TEST

In [ ]:
# output contains lists for all the prune_quantile values of the corresponding results
output = {"length":[],
          "coverage": [],
          "directness": [],
          "poirailway_coverage": [],
          "components": []
         }

for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
    if debug: print(prune_quantile, len(GT.vs))
    
    metrics = calculate_metrics(GT, G_carall, nnids, buffer_walk, numnodepairs)
    
    output["length"].append(metrics["length"])
    output["coverage"].append(metrics["coverage"])
    output["poirailway_coverage"].append(metrics["poi_coverage"])
    output["components"].append(metrics["components"])
    output["directness"].append(metrics["directness"])

In [ ]:
if debug:
    pp.pprint(output)
    
    fig = plt.figure(figsize=(4, 3)) # create figure object with a (width,height)
    axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
    y, x = cov.exterior.coords.xy
    patch1 = matplotlib.patches.Polygon(np.stack((np.asarray(x), -1*np.asarray(y)), axis=-1))
    axes.add_patch(patch1)
    axes.plot()

## Write to CSV

In [ ]:
filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".csv"

with open(PATH["results"] + filename, 'w') as f:
    w = csv.writer(f)
    w.writerow(output.keys())
    w.writerows(zip(*output.values()))